In [21]:
# basics
import sys
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import cv2
import pickle 
from PIL import Image
import shutil
# Keras Libraries
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.utils import  load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import keras.backend
import os
import random
import time
import datetime
from tensorflow.python.framework import ops
import inspect
import gc

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
accuracies=[]

In [23]:
batch_size = 14
target_size= (300,600)
# Define the path to the 5 folds
folders = ["/content/gdrive/MyDrive/Dataset/data_fold4","/content/gdrive/MyDrive/Dataset/data_fold3", "/content/gdrive/MyDrive/Dataset/data_fold2","/content/gdrive/MyDrive/Dataset/data_fold1","/content/gdrive/MyDrive/Dataset/data_fold0"]
model_name='mobilenet'

In [24]:



# Initialize a list to store the accuracy scores


# Iterate over the 5 folds
for fold in folders:

    fold_name = fold.split("/")[-1]
    model_path="/content/gdrive/MyDrive/Dataset/output/models/Modelmax+min__{}_{}.h5".format(model_name,fold_name)
    train_path = fold + r"/train"
   
    test_path = fold + r"/test"

    # Define the data generator
    datagen = ImageDataGenerator( rescale=1.0/255.0,
                                   shear_range = 0.02,
                                   zoom_range = 0.02, rotation_range=10,
                                  fill_mode='nearest',
                                  
                                  
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

    # Define the generator that will read images from the fold directory and their labels
    
    test_generator = datagen.flow_from_directory(directory = test_path, 
                                                    color_mode="rgb",
                                                        target_size = target_size, # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)

    test_generator.reset()

    model=tf.keras.models.load_model(model_path)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.1),loss='categorical_crossentropy',metrics=['accuracy'])
    accuracy = model.evaluate(test_generator)
    accuracies.append(accuracy)
    print(accuracy)
    print("Accuracy: {:.4f}%".format(accuracy[1] * 100))
    print("Loss: ",accuracy[0])




Found 56 images belonging to 4 classes.
4/4 [==============================] - 11s 2s/step - loss: 1.1458 - accuracy: 0.5714
[1.1457890272140503, 0.5714285969734192]
Accuracy: 57.1429%
Loss:  1.1457890272140503
Found 54 images belonging to 4 classes.
4/4 [==============================] - 10s 2s/step - loss: 0.9506 - accuracy: 0.5741
[0.9505614638328552, 0.5740740895271301]
Accuracy: 57.4074%
Loss:  0.9505614638328552
Found 84 images belonging to 4 classes.
6/6 [==============================] - 15s 2s/step - loss: 0.9771 - accuracy: 0.5952
[0.9770656824111938, 0.5952380895614624]
Accuracy: 59.5238%
Loss:  0.9770656824111938
Found 66 images belonging to 4 classes.
5/5 [==============================] - 12s 2s/step - loss: 0.7670 - accuracy: 0.6818
[0.7669796347618103, 0.6818181872367859]
Accuracy: 68.1818%
Loss:  0.7669796347618103
Found 98 images belonging to 4 classes.
7/7 [==============================] - 18s 2s/step - loss: 0.7712 - accuracy: 0.7449
[0.7712336778640747, 0.74489796

In [25]:
df = pd.DataFrame(accuracies)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/eval/accuracy_{}.csv'.format(model_name), index=False)

In [26]:
accuracies=np.array(accuracies)[:,1]
# Print the mean accuracy
print("Mean Accuracy: {:.2f}%".format(accuracies.mean() * 100))

Mean Accuracy: 63.35%


In [27]:
R=[]
import tensorflow as tf
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision(y_pred, y_true):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f1_score= 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
    return f1_score




In [28]:

for fold in folders:

    fold_name = fold.split("/")[-1]
    model_path="/content/gdrive/MyDrive/Dataset/output/models/Modelmax+min__{}_{}.h5".format(model_name,fold_name)
    
   
    test_path = fold + r"/test"

    # Define the data generator
   
        

    datagen = ImageDataGenerator( rescale=1.0/255.0,
                                      shear_range = 0.02,
                                      zoom_range = 0.02, rotation_range=10,
                                      fill_mode='nearest',
                                      horizontal_flip=True,
                                      
                                      width_shift_range=0.1,
                                      height_shift_range=0.1)

        
    test_generator = datagen.flow_from_directory(directory = test_path , 
                                                        color_mode="rgb",
                                                            target_size = target_size, # image height , image width
                                                            class_mode="categorical",
                                                            batch_size=batch_size,
                                                            shuffle=True,
                                                                seed=42)
    # load the saved model
    custom_objects = {'precision': precision,'recall': recall,'f1_score': f1_score}
    model = tf.keras.models.load_model(model_path,custom_objects=custom_objects)

    # compile the model with appropriate metrics

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',precision,recall,f1_score])

    # evaluate the model on the test data
    results = model.evaluate(test_generator,verbose=1)

    # create a dictionary to store the evaluation results
    metrics = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
    results_dict = {metrics[i]: results[i] for i in range(len(metrics))}
    R.append(results_dict)
    # convert the dictionary to a pandas dataframe
    results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['Value'])

    # print the evaluation results table
    print(results_df)

Found 56 images belonging to 4 classes.
4/4 [==============================] - 11s 2s/step - loss: 1.1318 - accuracy: 0.5714 - precision: 0.4643 - recall: 0.4643 - f1_score: 0.4643
              Value
Loss       1.131809
Accuracy   0.571429
Precision  0.464286
Recall     0.464286
F1-Score   0.464286
Found 54 images belonging to 4 classes.
4/4 [==============================] - 11s 2s/step - loss: 0.9760 - accuracy: 0.5926 - precision: 0.4464 - recall: 0.4464 - f1_score: 0.4464
              Value
Loss       0.976033
Accuracy   0.592593
Precision  0.446429
Recall     0.446429
F1-Score   0.446429
Found 84 images belonging to 4 classes.
6/6 [==============================] - 15s 2s/step - loss: 0.9677 - accuracy: 0.6071 - precision: 0.4405 - recall: 0.4405 - f1_score: 0.4405
              Value
Loss       0.967660
Accuracy   0.607143
Precision  0.440476
Recall     0.440476
F1-Score   0.440476
Found 66 images belonging to 4 classes.
5/5 [==============================] - 12s 2s/step - loss

In [29]:
df = pd.DataFrame(R)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/eval/stats_{}.csv'.format(model_name), index=False)

# method2 eval

In [ ]:
R1=[]

In [ ]:
# Define the path to the 5 folds
folders = ["/content/gdrive/MyDrive/Dataset/data_fold4","/content/gdrive/MyDrive/Dataset/data_fold3", "/content/gdrive/MyDrive/Dataset/data_fold2","/content/gdrive/MyDrive/Dataset/data_fold1","/content/gdrive/MyDrive/Dataset/data_fold0"]

# Initialize a list to store the accuracy scores


# Iterate over the 5 folds
for fold in folders:

    fold_name = fold.split("/")[-1]
    fold_name = fold_name.split("_")[-1]
    model_path="/content/gdrive/MyDrive/Dataset/output/models/model_tuned_{}_{}.h5".format(model_name,fold_name)
    train_path = fold + r"/train"
   
    test_path = fold + r"/test"

    # Define the data generator
    datagen = ImageDataGenerator( rescale=1.0/255.0,
                                   shear_range = 0.02,
                                   zoom_range = 0.02, rotation_range=10,
                                  fill_mode='nearest',
                                  
                                  
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

    # Define the generator that will read images from the fold directory and their labels
    
    test_generator = datagen.flow_from_directory(directory = test_path, 
                                                    color_mode="rgb",
                                                        target_size = (240,240), # image height , image width
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42)

    test_generator.reset()

    custom_objects = {'precision': precision,'recall': recall,'f1_score': f1_score}
    model = tf.keras.models.load_model(model_path,custom_objects=custom_objects)

    # compile the model with appropriate metrics

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',precision,recall,f1_score])

    # evaluate the model on the test data
    results = model.evaluate(test_generator,verbose=1)

    # create a dictionary to store the evaluation results
    metrics = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
    results_dict = {metrics[i]: results[i] for i in range(len(metrics))}
    R.append(results_dict)
    # convert the dictionary to a pandas dataframe
    results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['Value'])

    # print the evaluation results table
    print(results_df)


Found 56 images belonging to 4 classes.
4/4 [==============================] - 13s 3s/step - loss: 1.2700 - accuracy: 0.6071 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00
              Value
Loss       1.269993
Accuracy   0.607143
Precision  0.000000
Recall     0.000000
F1-Score   0.000000
Found 54 images belonging to 4 classes.
4/4 [==============================] - 14s 3s/step - loss: 1.7190 - accuracy: 0.2963 - precision: 0.1458 - recall: 0.1458 - f1_score: 0.1458
              Value
Loss       1.719028
Accuracy   0.296296
Precision  0.145833
Recall     0.145833
F1-Score   0.145833
Found 84 images belonging to 4 classes.
6/6 [==============================] - 19s 3s/step - loss: 1.4167 - accuracy: 0.2738 - precision: 0.0952 - recall: 0.0952 - f1_score: 0.0952
              Value
Loss       1.416685
Accuracy   0.273810
Precision  0.095238
Recall     0.095238
F1-Score   0.095238
Found 66 images belonging to 4 classes.
5/5 [==============================] - 19s 3s

In [ ]:
df = pd.DataFrame(R1)
df.to_csv('/content/gdrive/MyDrive/Dataset/output/eval/stats_tuned_{}.csv'.format(model_name), index=False)